In [ ]:
grouped = data.groupby('user_id')


In [ ]:
def create_sequences(data, window_size, target_cols):
    sequences = []
    labels = []
    for _, group in data.groupby('user_id'):  # Группировка по пользователям
        group = group.reset_index(drop=True)
        for i in range(len(group) - window_size):
            seq = group.iloc[i:i + window_size].drop(columns=['user_id'] + target_cols)
            label = group.iloc[i + window_size][target_cols].values
            sequences.append(seq.values)
            labels.append(label)
    return np.array(sequences), np.array(labels)

# Пример вызова
target_cols = ['attack', 'anomaly']  # Несколько целевых колонок
X, y = create_sequences(data, window_size=10, target_cols=target_cols)


In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(RNNModel, self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)  # output_dim = len(target_cols)
    
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Последний временной шаг
        return out

# Инициализация модели
input_dim = X.shape[2]
hidden_dim = 64
output_dim = len(target_cols)  # Количество целевых колонок
num_layers = 2

model = RNNModel(input_dim, hidden_dim, output_dim, num_layers)


In [ ]:
criterion = nn.BCEWithLogitsLoss()  # Пример для бинарных меток

def compute_loss(outputs, targets):
    return criterion(outputs, targets)

# Пример обучения
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_model(model, X_train, y_train, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        inputs = torch.tensor(X_train, dtype=torch.float32)
        targets = torch.tensor(y_train, dtype=torch.float32)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = compute_loss(outputs, targets)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train_model(model, X_train, y_train, criterion, optimizer)


In [ ]:
from sklearn.metrics import roc_auc_score

y_pred = model(torch.tensor(X_test, dtype=torch.float32)).sigmoid().detach().numpy()

# AUC-ROC для каждой метки
for i, col in enumerate(target_cols):
    auc = roc_auc_score(y_test[:, i], y_pred[:, i])
    print(f'{col}: AUC-ROC = {auc}')
